# vid2cleantxt

A basic example of how to use the [vid2cleantxt](https://github.com/pszemraj/vid2cleantxt) repo.


Use this as a base to start your own customized notebook by `File > Save a copy' and adjust it for your use case, i.e. add Google Drive integration, etc.

---


## setup


In [ ]:
#@markdown add auto-Colab formatting with `IPython.display`
from IPython.display import HTML, display
# colab formatting
def set_css():
    display(
        HTML(
            """
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """
        )
    )

get_ipython().events.register("pre_run_cell", set_css)

In [ ]:
#@markdown setup logging
import logging
from pathlib import Path
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

_log_location = Path.cwd() / "LOGFILE_vid2cleantxt_transcriber.log"

logging.basicConfig(
    level=logging.INFO,
    filename=_log_location,
    filemode='a',
    format="%(asctime)s %(levelname)s %(message)s",
    datefmt="%m/%d/%Y %I:%M:%S",
)
print(f'logfile is at:\n\n{_log_location}')

logfile is at:

/content/LOGFILE_vid2cleantxt_transcriber.log


In [ ]:
!nvidia-smi

Sun Feb 23 03:43:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%%time
pip install git+https://github.com/pszemraj/vid2cleantxt.git -q


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## import & download source video



This demo downloads a **video file** with `wget` to the local runtime and transcribes it. If you'd like to use this for youtube transcription, you can copy/paste the following code to download a video with [yt-dlp](https://github.com/yt-dlp/yt-dlp)  :



```python
!pip install -U -q yt-dlp
my_yt_link = "https://youtu.be/_UiL7ErcPf0"
!yt-dlp $my_yt_link --recode-video mp4
# video saved to notebook cwd, ready to run vid2cleantxt
```

Make sure that you run the youtube download code **before** running `vid2cleantxt.transcribe.transcribe_dir()`


---


In [ ]:
#@markdown enter a downloadable URL to a video
vid_file_url = "https://www.dropbox.com/s/q2gu4ik4h8zltpo/President%20Kennedy%27s%201962%20Speech%20on%20the%20US%20Space%20Program%20%20C-SPAN%20Classroom.mp4?dl=1" #@param {type:"string"}

import re

def extract_filename(url:str):
    base_name = re.findall(r'[^/]+$', url)[0]
    base_name =base_name.replace('?dl=0', "")
    base_name =base_name.replace('?dl=1', "")
    return re.sub('[^a-zA-Z0-9.]', '', base_name)

local_path = extract_filename(vid_file_url)
!wget -O $local_path $vid_file_url

--2023-01-25 19:04:28--  https://www.dropbox.com/s/q2gu4ik4h8zltpo/President%20Kennedy%27s%201962%20Speech%20on%20the%20US%20Space%20Program%20%20C-SPAN%20Classroom.mp4?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/q2gu4ik4h8zltpo/President%20Kennedy%27s%201962%20Speech%20on%20the%20US%20Space%20Program%20%20C-SPAN%20Classroom.mp4 [following]
--2023-01-25 19:04:29--  https://www.dropbox.com/s/dl/q2gu4ik4h8zltpo/President%20Kennedy%27s%201962%20Speech%20on%20the%20US%20Space%20Program%20%20C-SPAN%20Classroom.mp4
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfa85c90c9c256528d467ec32d7.dl.dropboxusercontent.com/cd/0/get/B1Nt1Grfuly0NCTK3Xw2n6JK64iiDBCHJElgQniqdPuEu6WUyKw6q5ajw2-go-Rvz-YY8hM78It35Ku7wx3nb0gev4gCUB1vlPrzCgH

In [ ]:
import vid2cleantxt

data folder is set to `/usr/local/lib/python3.11/dist-packages/neuspell/../data` script


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## print out the docstring for API usage

- `vid2cleantxt.transcribe.transcribe_dir` is the primary function at time of writing
- colab will also display this upon hovering over the function

In [ ]:
import inspect

deets =inspect.getfullargspec(vid2cleantxt.transcribe.transcribe_dir)
deets._asdict()

{'args': ['input_dir',
  'chunk_length',
  'model_id',
  'basic_spelling',
  'move_comp',
  'join_text',
  'print_memory_usage',
  'verbose'],
 'varargs': None,
 'varkw': None,
 'defaults': (30, 'openai/whisper-base.en', False, False, False, False, False),
 'kwonlyargs': [],
 'kwonlydefaults': None,
 'annotations': {'input_dir': str, 'chunk_length': int, 'model_id': str}}

## transcribe a video


- passing `"."` transcribes all videos in the current working directory



In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
text_output, metadata_output = vid2cleantxt.transcribe.transcribe_dir(
    input_dir=".", # notebook current working directory
    model_id='openai/whisper-small.en',
    chunk_length=30,
    join_text=True, # merge all transcribed text to one line (for demo outputs)
)


/usr/local/lib/python3.11/dist-packages/neuspell/seq_modeling/sclstmbert.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_data = torch.load(os.path.join(check

transcribing...:   0%|          | 0/1 [00:00<?, ?it/s]

Creating .wav audio clips:   0%|          | 0/275 [00:00<?, ?it/s]

Transcribing video:   0%|          | 0/275 [00:00<?, ?it/s]

SC_pipeline - transcribed audio:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# final transcription files are saved to this folder
text_output

'/content/v2clntxt_transcriptions/results_SC_pipeline'

In [ ]:
# final metadata files are saved to this folder
metadata_output

'/content/v2clntxt_transc_metadata'

In [ ]:
#@markdown display outputs
from pathlib import Path
output_dir = Path(text_output)
files = [f for f in output_dir.iterdir() if f.suffix == '.txt']
with open(files[0], 'r') as f:
    text = f.read()
text

"Music. You. All right so this is cs50 and this is week one zero index so to speak and it's not every day that you can say that you've learned a new language but today is that day today we explore a more traditional and older language called a and rest assured that even if what you're about to see no pun intended looks very cryptic very unusual particularly if you're among those less comfortable cling to the ideas from last week week zero wherein we talked. Talked about some of those fundamentals of functions and loops and conditionals all of which are coming back today indeed whereas last week and with problem set zero we focused on learning how to program with scratch which again you might have played with as a younger student days back today we focus on a instead but along the way we're going to focus as always frankly on learning how to solve problems but among the goals for today and really an entire class like this is just to give you week after week all the more tools for your t

In [ ]:
outpath = str(files[0])
outpath

NameError: name 'files' is not defined

## export outputs from Colab

- for summarization/other analytics, or printing and hanging on your wall
- if you had transcribed multiple files, you can zip and download the exports via


```python
from google.colab import files
archive_name = "my_transcriptions.zip"
# zip the returned `text_output` from vid2cleantxt
!zip -r -9 -j $archive_name $text_output
files.download(archive_name)
# wait for colab to download
```


for some examples of summarization, see the [textsum](https://github.com/pszemraj/textsum) python package, which is a utility for easily running summarization on strings/files with models such as [this one](https://huggingface.co/pszemraj/long-t5-tglobal-base-16384-book-summary)

---



In [ ]:
from google.colab import files
archive_name = "my_transcriptions.zip"
# zip the returned `text_output` from vid2cleantxt
!zip -r -9 -j $archive_name $text_output
files.download(archive_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>